# Figure 2
- Py Kernel with rpy2 integration for R and Python

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import sys
sys.path.insert(1, '/Users/brinkley97/Documents/development/')
import utils
import re

# Load Data

In [2]:
path_to_data =  "lab-kcad/datasets/tiles_dataset/" 
work_data = "figure_2/physical/diurnal_work_lm_6.csv.gz"
off_data = "figure_2/physical/diurnal_off_lm_6.csv.gz"

In [3]:
work_df = utils.load_gzip_csv_data(path_to_data, work_data)
off_df = utils.load_gzip_csv_data(path_to_data, off_data)

In [ ]:
# work_df.keys(), work_df

In [ ]:
# off_df

# Modify Dataframes

In [92]:
def modify_df(work_df, off_df):
    '''Modify Dataframes columns and values to match/flow with specific questions
    
    Arguments:
    work_df -- Dataframe
    off_df -- Dataframe
    
    Return: combined -- Dataframe
    '''
    
    
    # 0 is actually 11pm - 3am, ..., 5 is 7pm - 11pm
    # day starts with 2, which is 7am -11am, night starts with 5, which is 7pm - 11pm
    day_map = {'4th': "11:00–15:00 h", '5th': "15:00–19:00 h", '6th': "19:00–23:00 h", '1st': "23:00–3:00 h", '2nd': "3:00–7:00 h", '3rd': "7:00–11:00 h"}
    night_map = {'1st': '23:00–3:00 h', '2nd': '3:00–7:00 h', '3rd': '7:00–11:00 h', '4th': '11:00–15:00 h', '5th': '15:00–19:00 h', '6th': '19:00–23:00 h'}
    # shift_map = {'day': 'Day Shift', 'night': 'Night Shift'}
    
    
    work_df_copy = work_df.copy()
    work_df_copy = work_df_copy.drop(columns=["id", "age",  "gender"])
    update_work_df = work_df_copy.replace({"time": day_map})
    # update_work_df["rest"] = update_work_df["rest"] * 100
    # update_work_df["step_ratio"] = update_work_df["step_ratio"] * 100
    update_work_df["run_ratio"] = update_work_df["run_ratio"] * 100
    # update_work_df = update_work_df.reset_index(drop=True)
    # update_work_df = pd.DataFrame(index=[])
    
    
    # update_work_df = update_work_df.replace({"shift": shift_map})
    
    off_df_copy = off_df.copy()
    off_df_copy = off_df_copy.drop(columns=["id", "age",  "gender"])
    update_off_df = off_df_copy.replace({"time": night_map})
    # update_off_df = update_off_df.reset_index(drop=True)
    # update_off_df = update_off_df.replace({"shift": shift_map})
    # update_off_df = update_off_df["rest"] * 100
    
    # update_off_df["rest"] = update_off_df["rest"] * 100
    # update_off_df["step_ratio"] = update_off_df["step_ratio"] * 100
    # update_off_df["run_ratio"] = update_off_df["run_ratio"] * 100
    
    combine_dfs = pd.concat([update_work_df, update_off_df])
    combine_dfs = combine_dfs.reset_index(drop=True)
    return update_work_df, update_off_df, combine_dfs

In [93]:
# modified_dfs = modify_df(work_df, off_df)
# modified_dfs_copy = modified_dfs.copy()
# modified_dfs

In [94]:
# modified_dfs_copy

In [95]:
modified_work_df, modified_off_df, combine_dfs = modify_df(work_df, off_df)
# tmp_day_df.loc[tmp_day_df[‘type’] == col][‘data’]

In [65]:
# modified_work_df

,work,time,rest,step_ratio,run_ratio,shift
work,work,11:00–15:00 h,0.870900,0.294175,23.679036,night
work,work,19:00–23:00 h,0.628179,0.492380,40.210709,night
work,work,23:00–3:00 h,0.901501,0.477633,37.780797,night
work,work,3:00–7:00 h,0.948894,0.374328,28.810520,night
work,work,15:00–19:00 h,0.764789,0.324393,27.152216,night
...,...,...,...,...,...,...
work,work,19:00–23:00 h,0.572392,0.268371,16.266553,day
work,work,23:00–3:00 h,0.788603,0.169805,9.831218,day
work,work,3:00–7:00 h,0.986683,0.003109,0.119596,day
work,work,15:00–19:00 h,0.610316,0.271353,17.274722,day


In [90]:
# d = modified_work_df.reset_index(drop=True)

In [91]:
# f = d.nlargest(30, ["rest"])
# f

,work,time,rest,step_ratio,run_ratio,shift
9,work,3:00–7:00 h,1.000000,0.009845,0.393798,day
27,work,3:00–7:00 h,1.000000,0.019470,0.988080,day
81,work,3:00–7:00 h,1.000000,0.010585,0.355799,day
303,work,3:00–7:00 h,1.000000,0.009166,0.418410,day
477,work,3:00–7:00 h,1.000000,0.019155,0.992779,day
213,work,3:00–7:00 h,0.999780,0.007341,0.308302,day
585,work,3:00–7:00 h,0.999656,0.011220,0.463644,day
141,work,3:00–7:00 h,0.999467,0.012435,0.575528,day
309,work,3:00–7:00 h,0.998954,0.009879,0.441753,day
93,work,3:00–7:00 h,0.998685,0.006571,0.218812,day


In [81]:
# a = work_df.reset_index(drop=True)
# a

,work,time,rest,step_ratio,run_ratio,id,age,gender,shift
0,work,3rd,0.870900,0.294175,0.236790,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
1,work,5th,0.628179,0.492380,0.402107,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
2,work,6th,0.901501,0.477633,0.377808,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
3,work,1st,0.948894,0.374328,0.288105,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
4,work,4th,0.764789,0.324393,0.271522,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
...,...,...,...,...,...,...,...,...,...
607,work,5th,0.572392,0.268371,0.162666,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
608,work,6th,0.788603,0.169805,0.098312,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
609,work,1st,0.986683,0.003109,0.001196,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
610,work,4th,0.610316,0.271353,0.172747,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day


In [84]:
# a.loc[0:, ["time", "rest"]]

,time,rest
0,3rd,0.870900
1,5th,0.628179
2,6th,0.901501
3,1st,0.948894
4,4th,0.764789
...,...,...
607,5th,0.572392
608,6th,0.788603
609,1st,0.986683
610,4th,0.610316


In [89]:
# b = a.nlargest(30, ["rest"])
# b

,work,time,rest,step_ratio,run_ratio,id,age,gender,shift
9,work,1st,1.000000,0.009845,0.003938,02b7a595-6508-46bd-8239-6deb433d6290,< 40 Years,Female,day
27,work,1st,1.000000,0.019470,0.009881,0adb7679-9d26-46e7-a134-11da293910f3,>= 40 Years,Male,day
81,work,1st,1.000000,0.010585,0.003558,23a4808f-4429-41d7-adcf-c96742871613,< 40 Years,Female,day
303,work,1st,1.000000,0.009166,0.004184,89ff7373-ee85-4764-a4dd-90138af9af85,< 40 Years,Female,day
477,work,1st,1.000000,0.019155,0.009928,c7acafcf-f5dd-411e-b90c-988279852fb8,< 40 Years,Female,day
213,work,1st,0.999780,0.007341,0.003083,674e9815-7dd1-47f9-9916-c6e53d7e1a36,< 40 Years,Female,day
585,work,1st,0.999656,0.011220,0.004636,f610ffea-f6cb-4182-bbe7-19ff8fbe66ee,< 40 Years,Female,day
141,work,1st,0.999467,0.012435,0.005755,36d8aa27-272b-460f-8f2c-13d22f7150a5,< 40 Years,Female,day
309,work,1st,0.998954,0.009879,0.004418,8a01ec4c-6222-4efd-9beb-0adb12e00a2b,< 40 Years,Female,day
93,work,1st,0.998685,0.006571,0.002188,2a8f6d94-88d4-4d72-b304-782563903dca,< 40 Years,Female,day


In [83]:
# day_df = modified_work_df.loc[modified_work_df['shift'] == 'Day Shift']
# night_df = modified_work_df.loc[plt_df['shift'] == 'Night Shift']

In [ ]:
# combine_dfs

In [ ]:
# a = combine_dfs.loc[0:, ["work", "time", "rest", "shift"]]
# print(a)
# work_df = combine_dfs["work"] == "work"
# print(work_df)
# night_df = combine_dfs["work"] == "night"
# print(night_df)
# day_df = combine_dfs["shift"] == "day"
# print(day_df)
# night_df = combine_dfs["shift"] == "night"
# print(night_df)
# # by_time = combine_dfs.set_index("time")
# # print(by_time)
# by_time_group = combine_dfs.groupby(["time"]).mean()
# print(by_time_group)


# Load Generated Specific Questions

In [6]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
path_to_questions = "tiles-day-night/my_code"
name_of_questions = "generateSpecificQuestions.ipynb"
questions = base + path_to_questions + name_of_questions

In [7]:
# %load questions
/Users/brinkley97/Documents/development/lab-kcad/tiles-day-night/my_codegenerateSpecificQuestions.ipynb

In [8]:
%run "generateSpecificQuestions.ipynb"

In [9]:
figure_2_specific_questions = fig_2

# Py + R Code Integration

In [ ]:
# %load_ext rpy2.ipython

In [ ]:
# import rpy2.robjects as robjects
# import rpy2.robjects.packages as rpackages

# # load and activate() bc if NOT error (Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>') will appear
# from rpy2.robjects import pandas2ri
# pandas2ri.activate()

In [ ]:
# report = rpackages.importr('report')

In [ ]:
# rpackages.importr('devtools')
# rpackages.importr('ez')

In [ ]:
# def specify_work_or_off(modified_work_df, modified_off_df, combined_dfs):
#     df_columns = list(combined_dfs.keys())
#     # print(df_columns)
    
#     filter_physiologicals = list(key_words_in_study.values())[6]
#     filter_times = list(key_words_in_study.values())[7]
    
#     # print(filter_physiologicals, filter_times)
#     dfs = []
    
#     group_by_time_work_df = modified_work_df.set_index(["time"])
#     # print(group_by_time_work_df)
    
#     group_by_time_off_df = modified_off_df.set_index(["time"])
#     # print(group_by_time_off_df)
    
#     for time in filter_times:
#         '''
#         Work
#         '''
#         work_by_time = group_by_time_work_df.loc[time]
#         # print(time, "\n", work_by_time, "---")
#         index_by_work = work_by_time.set_index(["work"])
#         print(time, "\n", index_by_work, "---")
        
        # shift_day = index_by_work.loc[index_by_work["shift"] == "day"]
        # print(time, "\n", "work", "day", shift_day, "---")
        
        # shift_night = index_by_work.loc[index_by_work["shift"] == "night"]
        # print(time, "\n", "work", "night", shift_night, "---")
        
#         step_day = shift_day["step_ratio"]
#         print(time, "\n", "day", step_day)
        
#         step_night = shift_night["step_ratio"]
#         print(time, "\n", "night", step_night)
        
        # '''
        # Off
        # '''
#         off_by_time = group_by_time_off_df.loc[time]
#         # print(time, "\n", off_by_time, "---")
#         index_by_off = off_by_time.set_index(["work"])
#         # print(time, "\n", index_by_off, "---")

#         shift_day = index_by_off.loc[index_by_off["shift"] == "day"]
#         # print(time, "\n", "work", "day", shift_day, "---")
        
#         shift_night = index_by_off.loc[index_by_off["shift"] == "night"]
#         # print(time, "\n", "work", "night", shift_night, "---")
        
#         step_day = shift_day["step_ratio"]
#         print(time, "\n", "day", step_day)
        
#         step_night = shift_night["step_ratio"]
#         print(time, "\n", "night", step_night)


    

In [ ]:
# specify_work_or_off(modified_work_df, modified_off_df, combine_dfs)

In [129]:
def specific_questions_with_df(specific_question, specific_question_by_words, work_df, off_df, word_to_find):
    '''
    
    Arguments:
    specific_question -- string
    df -- Dataframe
    word_to_find -- string
    
    TODO:
    Split df by work or off
    Put all dfs by time togeter 102 rows each
    Multiply rest and step by 100 in a new col 
    
    '''
    
    print(specific_question)
    # print(rest_or_step)
    filter_shifts = ["day", "night"]
    filter_times = list(key_words_in_study.values())[7]
    p_val_df = pd.DataFrame(index=filter_times)
    # print(work_df)
    work_df = work_df.reset_index(drop=True)
    # print(work_df)
    
    group_by_time_work_df = work_df.set_index(["time"])
    # print(group_by_time_work_df)
    
    off_df = off_df.reset_index(drop=True)
    # print(off_df)
    
    group_by_time_off_df = off_df.set_index(["time"])
    
    for time_idx in range(len(specific_question_by_words)):
        # print(time_idx)
        time = specific_question_by_words[time_idx]
        if time in filter_times:
            
            '''
            Work DF
            '''
            work_by_time = group_by_time_work_df.loc[time]
            # print("\n", work_by_time, "---")
                
            shift_day = work_by_time.loc[work_by_time["shift"] == "day"]
            # print(time, "\n", "work", "day", shift_day)
                
            shift_night = work_by_time.loc[work_by_time["shift"] == "night"]
            # print(time, "\n", "work", "night", shift_night)
                
            
            stats_value, work_p = stats.ttest_ind(np.array(shift_day[word_to_find]), np.array(shift_night[word_to_find]))
            print("work", time, work_p)
            
            p_val_df.loc[time, "p-val"] = work_p
            
            '''
            Off DF
            '''
            off_by_time = group_by_time_off_df.loc[time]
            # print("\n", off_by_time, "---")
                
            off_shift_day = off_by_time.loc[off_by_time["shift"] == "day"]
            # print(time, "\n", "off", "day", off_shift_day)
                
            off_shift_night = off_by_time.loc[off_by_time["shift"] == "night"]
            # print(time, "\n", "off", "night", off_shift_night)
                
            
            stats_value, off_p = stats.ttest_ind(np.array(off_shift_day[word_to_find]), np.array(off_shift_night[word_to_find]))
            print("off", time, off_p)
            print("------------------------------------------")
            p_val_df.loc[time, "p-val"] = off_p
            

    return p_val_df

In [132]:
def figure_2(generated_specific_questions, work_df, off_df):
    '''Reproduce Table 2
    
    Arguments:
    generated_specific_questions -- list
    df -- Dataframe
    
    
    Return: specific_data -- Dataframe in list
    
    '''
    # print(type(generated_specific_questions), generated_specific_questions)
    # print(type(work_df), work_df)
    # print(type(off_df), off_df)

    # filter_by = filter_physiologicals + filter_time
    # print(filter_by)
    
    # for filter_physiological_idx in range(len(filter_physiologicals)):
    #     print()
    #     if filter_physiologicals[filter_physiological_idx] in work_df_columns:
    #         work_df['data'] = work_df.loc[0:, [filter_physiologicals[filter_physiological_idx], ]]
    #         # work_df['data'] = work_df['data'].set_axis(['work'])
    #         print(work_df['data'])
        
    # i = 0
    p_values = []
    filter_physiological = ["rest", "step_ratio"]
    for generated_specific_question_idx in range(len(generated_specific_questions)):
        specific_question = generated_specific_questions[generated_specific_question_idx]
        # print(generated_specific_question_idx, "th specific_question : ", specific_question)
        specific_question_by_words = specific_question.split("*")
        # print(specific_question_by_words)
        
#         for work_df_column in work_df_columns:
#             if work_df_column in specific_question.split("*"):
#                 print(work_df_column, True)
        work_df = work_df.reset_index(drop=True)
        
        for filter_physiological_idx in range(len(filter_physiological)):
            
            word_to_find = filter_physiological[filter_physiological_idx]
            if word_to_find in specific_question_by_words:
                # print(word_to_find)
                modified_work_df = work_df.loc[0:, ["work", "time", word_to_find, "shift"]]
                # print(generated_specific_question_idx, specific_question)
                p_val = specific_questions_with_df(specific_question, specific_question_by_words, modified_work_df, off_df, word_to_find)
                # print(p_val)
                # p_val_df.loc[filter_times[i], "p-val"] = p_val
                # i = i + 1
                # p_values.append(p_val)
                # print(p_val)
    return 

In [133]:
figure_2(figure_2_specific_questions, modified_work_df, modified_off_df)

on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *23:00–3:00 h* ?
work 23:00–3:00 h 0.0015494236117052733
off 23:00–3:00 h 0.8501148450822772
------------------------------------------
on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *3:00–7:00 h* ?
work 3:00–7:00 h 0.034467656157217745
off 3:00–7:00 h 0.3176447299854959
------------------------------------------
on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between the hours of *7:00–11:00 h* ?
work 7:00–11:00 h 8.619878000514936e-08
off 7:00–11:00 h 4.227081558573983e-05
------------------------------------------
on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *rest* between th

# Without R

In [ ]:
# import numpy as np

In [ ]:
# a = np.random.randn(40)
# print(a)
# b = 4*np.random.randn(50)
# print(b)

# # Use scipy.stats.ttest_ind.
# t, p = stats.ttest_ind(a, b, equal_var=False)
# print("ttest_ind:            t = %g  p = %g" % (t, p))
